<a href="https://colab.research.google.com/github/PrachiHarish/DL-Projects/blob/main/Copy_of_Copy_of_Copy_of_MLP_regression_DL_framework_boston_house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.layers import*
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import pandas as pd
path="/content/drive/MyDrive/Dataset/boston.csv"
dataset=pd.read_csv(path)
dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [ ]:
dataset.shape

(506, 14)

In [ ]:
dataset.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [ ]:
dataset.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [ ]:
import numpy as np
x=np.array(dataset.iloc[:,0:13])
y=np.array(dataset[["MEDV"]])
print(x.shape)
print(y.shape)

(506, 13)
(506, 1)


In [ ]:
# without using in-built lib
x_norm=(x-x.mean())/x.std()
y_norm=(y-y.mean())/y.std()
print(x_norm.shape)
print(y_norm.shape)

(506, 13)
(506, 1)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x_norm,y_norm,test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(404, 13)
(102, 13)
(404, 1)
(102, 1)


**Method 1: Sequential**

In [ ]:
model_ann1=Sequential()
model_ann1.add(Dense(16,input_dim=13,activation="relu",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="Hidden_1"))
model_ann1.add(Dense(32,activation="relu",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="Hidden_2"))
model_ann1.add(Dense(1,activation="linear",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="output"))
model_ann1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden_1 (Dense)            (None, 16)                224       
                                                                 
 Hidden_2 (Dense)            (None, 32)                544       
                                                                 
 output (Dense)              (None, 1)                 33        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


**Method 2: Functional**

In [ ]:
inp=Input(shape=(13,),name="Input")
h1=Dense(16,activation="relu",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="hidden_1")(inp)
h2=Dense(32,activation="relu",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="hidden_2")(h1)
output=Dense(1,activation="linear",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="Output")(h2)
model_ann3=Model(inp,output)
model_ann3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 13)]              0         
                                                                 
 hidden_1 (Dense)            (None, 16)                224       
                                                                 
 hidden_2 (Dense)            (None, 32)                544       
                                                                 
 Output (Dense)              (None, 1)                 33        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


**Method 3: Subclass**

In [ ]:
class CustomModel:
  def __init__ (self):
    self.dense1=Dense(16,activation="relu",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="Hidden_1")
    self.dense2=Dense(32,activation="relu",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="Hidden_2")
    self.output=Dense(1,activation="linear",kernel_initializer="he_uniform",bias_initializer="he_uniform",name="Output")
  def call(self,shape):
    self.inp=Input(shape,name="Input")
    h1=self.dense1(self.inp)
    h2=self.dense2(h1)
    output=self.output(h2)
    model=Model(self.inp,output)
    return model
m=CustomModel()
model_ann3=m.call(shape=(13,))
model_ann3.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 13)]              0         
                                                                 
 Hidden_1 (Dense)            (None, 16)                224       
                                                                 
 Hidden_2 (Dense)            (None, 32)                544       
                                                                 
 Output (Dense)              (None, 1)                 33        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_ann1.compile(optimizer="SGD",loss="mean_squared_error",metrics=["accuracy"])
history_ann1=model_ann1.fit(x_train,y_train,epochs=10,batch_size=4,validation_data=(x_test,y_test),verbose=1)


Epoch 1/10
101/101 [==============================] - 1s 4ms/step - loss: 1.0794 - accuracy: 0.0000e+00 - val_loss: 0.5882 - val_accuracy: 0.0000e+00
Epoch 2/10
101/101 [==============================] - 0s 2ms/step - loss: 0.8739 - accuracy: 0.0000e+00 - val_loss: 0.5180 - val_accuracy: 0.0000e+00
Epoch 3/10
101/101 [==============================] - 0s 2ms/step - loss: 0.8411 - accuracy: 0.0000e+00 - val_loss: 0.6701 - val_accuracy: 0.0000e+00
Epoch 4/10
101/101 [==============================] - 0s 2ms/step - loss: 0.8189 - accuracy: 0.0000e+00 - val_loss: 0.6340 - val_accuracy: 0.0000e+00
Epoch 5/10
101/101 [==============================] - 0s 2ms/step - loss: 0.8144 - accuracy: 0.0000e+00 - val_loss: 0.5970 - val_accuracy: 0.0000e+00
Epoch 6/10
101/101 [==============================] - 0s 2ms/step - loss: 0.7625 - accuracy: 0.0000e+00 - val_loss: 0.4927 - val_accuracy: 0.0000e+00
Epoch 7/10
101/101 [==============================] - 0s 2ms/step - loss: 0.7614 - accuracy: 0.0000e

In [ ]:
y_pred=model_ann1.predict(x_test)
print(y_pred)

4/4 [==============================] - 0s 2ms/step
[[-2.99560308e-01]
 [-2.85600305e-01]
 [-3.80291462e-01]
 [-3.80621552e-02]
 [ 2.49701142e-01]
 [-3.25444221e-01]
 [-3.75710249e-01]
 [-2.33532131e-01]
 [ 4.44669366e-01]
 [ 5.35869598e-03]
 [-6.76154017e-01]
 [-1.61801577e-02]
 [ 1.38751268e-02]
 [-1.88915193e-01]
 [-4.07057047e-01]
 [ 2.30036497e-01]
 [ 6.67452931e-01]
 [ 3.04020524e-01]
 [-3.69395494e-01]
 [-3.65440249e-01]
 [ 5.48715949e-01]
 [-3.77935529e-01]
 [-3.47357512e-01]
 [-8.24928284e-04]
 [-1.98055208e-01]
 [ 3.10317278e-02]
 [ 1.86903477e-01]
 [ 5.61285138e-01]
 [ 5.71083903e-01]
 [ 6.90429211e-02]
 [-3.74740362e-02]
 [-3.81717026e-01]
 [-7.67293811e-01]
 [-5.17504930e-01]
 [ 2.51694441e-01]
 [ 1.47339463e-01]
 [-4.97891307e-01]
 [-7.82456040e-01]
 [ 2.82412887e-01]
 [-2.10238218e-01]
 [-5.40942550e-01]
 [-4.12640512e-01]
 [-2.09276795e-01]
 [-3.75863671e-01]
 [ 2.42014408e-01]
 [ 2.56514072e-01]
 [ 2.74974108e-02]
 [-3.87730718e-01]
 [ 1.11459851e-01]
 [ 3.24700952e-01]

In [ ]:

from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from math import sqrt
print("MAE :" , mean_absolute_error(y_test,y_pred))
print("RMSE :" , np.sqrt(mean_squared_error(y_test,y_pred)))


MAE : 0.48759334965554596
RMSE : 0.7073384739959557
